## My aim was do dos some exploratory data analysis on this dataset and we could try to predict if a pokemon was legendary or not so i've done that as well..

### imports

In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

from itertools import combinations
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, classification_report

In [ ]:
# download the dataset from here
# https://www.kaggle.com/ahmettezcantekin/beginner-datasetss

df = pd.read_csv("pokemon.csv")
df.head(10)

## EDA

In [ ]:
df.info()

# null values in type 2 column...

# i think fillna can handle this!

In [ ]:
df['Type 2'].fillna(df['Type 1'], inplace=True)

In [ ]:
df.head()

In [ ]:
# random idea can you predict the type of a pokemon given its stats...?

In [ ]:
plt.figure(figsize=(10,8))
ax = sns.barplot(df['Type 1'].value_counts().values, df['Type 1'].value_counts().index, orient='h');
ax.set(xlabel="Number of pokemon as per type 1");

In [ ]:
plt.figure(figsize=(10,8))
ax = sns.barplot(df['Type 2'].value_counts().values, df['Type 2'].value_counts().index, orient='h');
ax.set(xlabel="Number of pokemon as per type 2");

In [ ]:
# im not getting any clear ideas..
# lets see correlation to get some if possible

plt.figure(figsize=(10,8))
cor = df.corr()
sns.heatmap(cor,annot=True,cmap='Greens');

### Ledendary shows some correlation... and some ppl have tried to predict legendary.

### I will use a model on it but for now i'm gonna try and extract some data for exploration purpose

In [ ]:

plt.figure(figsize=(10,8))
ax = sns.barplot(df.groupby(['Type 1'])['Attack'].mean().values, df.groupby(['Type 1'])['Attack'].mean().index, orient='h');
ax.set(xlabel="Attack as per Type 1");

In [ ]:
# we might want to combine type 1 and 2 to create a new type

all_types = pd.concat([df['Type 1'], df['Type 2']], ignore_index=True, axis=0).unique()
comb = combinations(all_types, 2)

type_combo = []
for i in list(comb):
    type_combo.append(' '.join(i))


def type_clf(row):
#     print(row)
    if (row['Type 1'] + ' ' + row['Type 2']) in type_combo: 
        return (row['Type 1'] + ' ' + row['Type 2'])
    elif (row['Type 2'] + ' ' + row['Type 1']) in type_combo:
        return (row['Type 2'] + ' ' + row['Type 1'])
    elif row['Type 1'] == row['Type 2']:
        return row['Type 1']
    
    
        
        
    
df['Type'] = df.apply(type_clf, axis=1)

In [ ]:
df.head()

In [ ]:

plt.figure(figsize=(10,25))
ax = sns.barplot(df.groupby(['Type'])['Attack'].mean().values, df.groupby(['Type'])['Attack'].mean().index, orient='h');
ax.set(xlabel="Attack as per Type 1");

In [ ]:
df.groupby(['Type']).agg({'Attack': ['mean', 'min', 'max']})

In [ ]:
plt.figure(figsize=(10,25))
ax = sns.barplot(df.groupby(['Type'])['HP'].mean().values, df.groupby(['Type'])['HP'].mean().index, orient='h');
ax.set(xlabel="HP as per Type 1");

In [ ]:
plt.figure(figsize=(10,25))
ax = sns.barplot(df.groupby(['Type'])['Defense'].mean().values, df.groupby(['Type'])['Defense'].mean().index, orient='h');
ax.set(xlabel="Defence as per Type 1");

In [ ]:
plt.figure(figsize=(10,25))
ax = sns.barplot(df.groupby(['Type'])['Speed'].mean().values, df.groupby(['Type'])['Speed'].mean().index, orient='h');
ax.set(xlabel="Speed as per Type 1");

### if you'd want to become a pokemon master, you clearly need to use this dataset

In [ ]:
# The total column represents the total of all the stats... lets see who's at the top and bottom

In [ ]:
df[df['Total'] == max(df['Total'])]

In [ ]:
df[df['Total'] == min
   (df['Total'])]

In [ ]:
# but i dont want legendardy for max

df[df['Total'] == max(df[df['Legendary'] == False]['Total'])][df['Legendary'] == False]

In [ ]:
# lets see the weakest legendary

df[df['Total'] == min(df[df['Legendary'] == True]['Total'])][df['Legendary'] == True]

In [ ]:
# does belonging to a higher generation make you stronger?

df.groupby(['Generation'])['Total'].mean()

# ok. 

In [ ]:
df.groupby(['Generation'])['Total'].max()

In [ ]:
df.groupby(['Generation'])['Total'].min()

In [ ]:
# getting all data. and i realized i can't see all of it...

df.groupby(['Type'])['Total','HP' ,'Attack','Defense','Sp. Atk','Sp. Def','Speed'].describe()

In [ ]:
df.head()

### Now, lets make the classificaition model

In [ ]:
for col in df.iloc[:,4:12].columns:
    sns.distplot(df[col])
    plt.show()

### most of our data if normalish distributed

In [ ]:

for col in df.iloc[:,4:12].columns:
    sns.distplot(np.sqrt(df[col]))    # sqrt usually doesn't have a big effect but log caused left skeweness
    plt.show()

In [ ]:
for col in df.iloc[:,4:12].columns:
    sns.distplot(np.log1p(df[col]))    # sqrt usually doesn't have a big effect but log caused left skeweness
    plt.show()

In [ ]:
# lets fix our data

for col in df.iloc[:,5:11].columns:
    df[col] = np.sqrt(df[col])   
    

In [ ]:
### except type, generation and '#' column

features = ['Total', 'HP', 'Attack', 'Defense',
       'Sp. Atk', 'Sp. Def', 'Speed']

X = df[features]
y = df['Legendary'].replace({True:1,False:0})

In [ ]:
y

### Standardize it

In [ ]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

### train test split... i really need to start using other stuff like stratified k fold, cv, etc,. instead

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=42)

### Models

In [ ]:
log_clf = LogisticRegression()

log_clf.fit(X_train,y_train)

log_clf.score(X_test,y_test)

In [ ]:
log_clf_cv = LogisticRegressionCV(cv=5, random_state=0)

log_clf_cv.fit(X_train,y_train)

log_clf.score(X_test,y_test)

In [ ]:
rf_clf = RandomForestClassifier()
rf_clf.fit(X_train,y_train)
rf_clf.score(X_test,y_test)

### Performance metrics

In [ ]:
df['Legendary'].value_counts()

In [ ]:
y_pred = log_clf.predict(X_test)

In [ ]:
confusion_matrix(y_test,y_pred)

# 6 false negative top right
# 6 false positive bottom left

In [ ]:
print(classification_report(y_test,y_pred))

# Thank you
## Don't forget to leave a like or upvote if it was worth your time.